In [1]:
import os
import pandas as pd
import numpy as np
# from openpyxl import load_workbook

In [2]:
# Initialize values for the parameters
source = r"C:\Users\zygo\Desktop\DataRecords\12-21-2019"
batchName = "A_Arena"
fileCount = 50

In [3]:
# Iterate over directories to search for .csv files, then append into 1 single list until specified file count 

def extract(source, batchName, fileCount):
    data=[]
    for chamberID in os.listdir(source):
        count=0
        file_path = os.path.join(source, chamberID, batchName)

        if os.path.isdir(file_path):
            for files in os.listdir(file_path):
                if files.endswith(".csv"):
                    new_file_path = os.path.join(file_path, files)
                    df = pd.read_csv(new_file_path)
                    data.append(df)
                    count+=1
                    if count >= fileCount:
                        break
    return data

In [4]:
# Clean and transform data into desirable format

def transform(data):
    # Concatenate all DataFrames horizontally along columns
    merged_data = pd.concat(data, axis=1)
    
    #Drop duplicated columns
    filtered_data = merged_data.T.drop_duplicates().drop('Data', axis=0)
    filtered_data = filtered_data.T.set_index('Time(ms)')
    
    # Drop rows where any column has zero values
    filtered_data = filtered_data.loc[(filtered_data != 0).all(axis=1)]
    
    #Rename columns
    column_list = []
    
    for chamber in range(18):
        chamber_id = chamber + 1
        for i in range(50):
            column_list.append("Chamber{}_{}".format(chamber_id, i+1)) 
    filtered_data.columns = column_list
    
    return filtered_data

In [15]:
# Calculate sum for specific rows and organize the sums accordingly to the columns in a DataFrame.

def sum(filtered_data):
    sum_data = filtered_data.iloc[12:23]
    sum_list = sum_data.sum().tolist()
    reshaped_sum_list = np.array(sum_list).reshape(-1, fileCount)
    
    column_sum_list = []
    for chamber in range(len(os.listdir(source))):
        column_sum_list.append("Chamber{}".format(chamber + 1))
    
    sum_data_ph = []
    sum_df = pd.DataFrame(sum_data_ph, columns=column_sum_list)
    
    for i in range(len(column_sum_list)):
        sum_df[column_sum_list[i]] = reshaped_sum_list[i].tolist()
    return sum_df

In [6]:
# Generate descriptive statistics for data obtained from sum function
def descriptive_stats(sum_data):
    dstats_df = sum_data.describe().round().astype(int)
    specified_dstats = ['max','min','mean','std']
    #dstats_output = dstats_df[dstats_df.index.isin(specified_dstats)]
    dstats_output = dstats_df.filter(items = specified_dstats, axis=0)
    return dstats_output

In [35]:
# Export file to excel
def load(data, sum_data, dstats_data):
    excel_file_path = r'C:\Users\zygo\Desktop\Report\New_Test_{}.xlsx'.format(fileCount)
    with pd.ExcelWriter(excel_file_path) as writer:
        data.to_excel(writer, sheet_name='Data')
        sum_data.to_excel(writer, sheet_name='Sum')
        dstats_data.to_excel(writer, sheet_name='DescriptiveStats')
    print(f"Files have been exported to excel.")

In [18]:
data = extract(source, batchName, fileCount)

In [20]:
dataframe = transform(data)

In [21]:
sum_data = sum(dataframe)

In [23]:
dstats = descriptive_stats(sum_data)

In [34]:
load(dataframe, sum_data, dstats)

Files have been exported to excel.


In [ ]:
# Set the path for the output CSV file
#output_path = r'C:\Users\zygo\Desktop\KP\test.csv'

# Write the merged data to a CSV file
#filtered_data.to_csv()
#print(r'Merged data saved to C:\Users\zygo\Desktop\KP')